# 1. Data and module import

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

In [11]:
dataset = pd.read_csv('../timeseries-forecasting/salesdata_ger.csv')
dataset.head(5)

,OrderDate,QtyOrdered
0,2019-01-01,9279
1,2019-01-02,11073
2,2019-01-03,12272
3,2019-01-04,12617
4,2019-01-05,13354


In [12]:
# Convert data into format required by Prophet

dataset_train = dataset.query("OrderDate <= '2021-12-31'")
df = pd.DataFrame({'ds': pd.to_datetime(dataset_train['OrderDate']), 'y': dataset_train['QtyOrdered']})
df.tail(5)

,ds,y
1091,2021-12-27,31431
1092,2021-12-28,29659
1093,2021-12-29,28661
1094,2021-12-30,24712
1095,2021-12-31,15377


In [13]:
# Declare black_fridays df to input as parameter to the model

black_fridays = pd.DataFrame({'holiday': 'Black Friday',
                              'ds': pd.to_datetime(['2019-11-29', '2020-11-27', '2021-11-25', '2022-11-25']),
                              'lower_window': 0,
                              'upper_window': 2})

# Initialize and fit Prophet model with holidays parameter
prophet = Prophet(holidays=black_fridays)
prophet.add_country_holidays(country_name='DE')
prophet.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [14]:
prophet.train_holiday_names

0                 Black Friday
1                      Neujahr
2                   Karfreitag
3                  Ostermontag
4                   Erster Mai
5          Christi Himmelfahrt
6                Pfingstmontag
7    Tag der Deutschen Einheit
8         Erster Weihnachtstag
9        Zweiter Weihnachtstag
dtype: object

In [ ]:
# Declare future timeline
